In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import metrics

In [ ]:
#directory = '/Users/filippo/Developer/tesi/results/hSBM/overlap_5000doc_0.25occ_fpkmweight_thr/topsbm'
#directory = '/Users/filippo/Developer/tesi/gtex/hsbm/nonoverlap05/topsbm'
directory = '/Users/filippo/Developer/tesi/gtex/hsbm/oversigma_5tissue'

In [ ]:
l = 3

In [ ]:
df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topics.csv"%(directory,l), index_col=None)
df_topics.head()

# topic size

In [ ]:
sizes = []
for t in df_topics.columns:
    sizes.append(len(df_topics.loc[:,t].dropna()))

In [ ]:
fig=plt.figure()
plt.hist(sizes, histtype='step', lw=2)
plt.xlabel("topic size (# genes)", fontsize=16)
plt.ylabel("# topic of that size", fontsize=16)
plt.show()
fig.savefig("topic_size_%d.png"%l)

## Topic O

In [ ]:
df_mv=pd.read_csv("meanVariances.csv", index_col=[0])
df_o = pd.read_csv("O.dat", header=None)
df_mv.insert(3, 'occurrence', df_o.values)
df_mv['occurrence'] = df_mv['occurrence']
df_mv.head()

In [ ]:
avgs = []
semidisps = []
for topic in df_topics.columns:
    subarr = df_mv.loc[df_topics[topic].dropna(),:]['occurrence'].values
    avg = np.average(subarr)
    std = np.std(subarr)
    avgs.append(avg)
    semidisps.append([avg-np.min(subarr),np.max(subarr)-avg])

In [ ]:
N = len(df_topics.columns)
fig = plt.figure(figsize=(15,20))
plt.errorbar(avgs, np.arange(N), xerr=np.array(semidisps).T, yerr=None, fmt='x', marker='x', lw=5, ms=50)
plt.xlabel('$O_i$', fontsize=18)
plt.yticks(np.arange(len(df_topics.columns)),df_topics.columns.values, fontsize=16)
plt.xticks(fontsize=18)
plt.xlim(0,1)
plt.show()
fig.savefig("%s/O_ditrs_l_%d.pdf"%(directory,l))

# Geneontology

In [ ]:
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol

In [ ]:
import importlib
importlib.reload(geneontology)

In [ ]:
#import gseapy as gs
#gs.get_library_name()

In [ ]:
back_sea=[]
df_world = pd.read_csv("%s/background.txt"%directory, header=None)
for g in df_world.values:
    back_sea.append(get_symbol(g[0]))

In [ ]:
ontology = []
for g in df_topics[df_topics.columns[2]].dropna():
    ontology.append(get_symbol(g))

In [ ]:
topic_pvalues = []

In [ ]:
for topic in df_topics.columns:
    symbols = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
    print(topic, " - " ,len(symbols))
    enriched_topic = get_ontology_df(symbols, background=back_sea).sort_values(by=['Adjusted P-value'], ascending=True)
    enriched_topic = enriched_topic.loc[enriched_topic.index.values[:20],:]
    enriched_topic.to_csv("%s/gsea_level_%d_topic%s.csv"%(directory,l,topic))
    print(enriched_topic)
    topic_pvalues.append(enriched_topic.loc[enriched_topic.index.values[0],'Adjusted P-value'])